In [ ]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Input Mono;
        line-height: 1.2 !important;
        font-size: 12px !important;
    }
</style>
""")

# Recommendations with IBM

In this notebook, you will be putting your recommendation skills to use on real data from the IBM Watson Studio platform. 


You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/rubrics/3325/view).  **Please save regularly.**

By following the table of contents, you will build out a number of different methods for making recommendations that can be used for different situations. 


## Table of Contents

I. [Exploratory Data Analysis](#Exploratory-Data-Analysis)<br>
II. [Rank Based Recommendations](#Rank)<br>
III. [User-User Based Collaborative Filtering](#User-User)<br>
IV. [Content Based Recommendations (EXTRA - NOT REQUIRED)](#Content-Recs)<br>
V. [Matrix Factorization](#Matrix-Fact)<br>
VI. [Extras & Concluding](#conclusions)

At the end of the notebook, you will find directions for how to submit your work.  Let's get started by importing the necessary libraries and reading in the data.

In [ ]:
import numpy as np, pandas as pd, plotly.express as px

df = pd.read_csv('data/user-item-interactions.csv').drop(columns='Unnamed: 0')
df_content = pd.read_csv('data/articles_community.csv').drop(columns='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
df_content.head()

### <a class="anchor" id="Exploratory-Data-Analysis">Part I : Exploratory Data Analysis</a>

Use the dictionary and cells below to provide some insight into the descriptive statistics of the data.

`1.` What is the distribution of how many articles a user interacts with in the dataset?  Provide a visual and descriptive statistics to assist with giving a look at the number of times each user interacts with an article.  

In [ ]:
df.shape, df_content.shape, df.article_id.nunique(), df.email.nunique(), df_content.article_id.nunique()

In [ ]:
# number of articles read be each user (identified via unique email address)
df_user_article_interaction = df.groupby('email').agg({'article_id': 'count'}).sort_values('article_id', ascending=False).reset_index().rename(columns={'article_id': 'article_count'})

In [ ]:
fig = px.bar(df_user_article_interaction, x='email', y='article_count', width=1300)
fig.update_layout(
    xaxis={'visible': True, 'showticklabels': False},
    font_family='Input Mono',
    font_color='black',
    title_x=0.50,
    title_y=0.95,
    title_text='Distribution of Number of Articles read by each User',
    xaxis_title='User (Identified by unique Email)',
    yaxis_title='Number of Articles read',
    plot_bgcolor='rgba(255,255,255,255)',
    paper_bgcolor='rgba(255,255,255,255)')
fig.show()

In [ ]:
# descriptive statistics of number of articles read be each user (identified via unique email address)
df_user_article_interaction.describe()

In [ ]:
# fill in the median and maximum number of user-article interactios below

median_val = df_user_article_interaction.article_count.median()
max_views_by_user = df_user_article_interaction.article_count.max()

print(f'50% of individuals interact with [{median_val}] number of articles or fewer')
print()
print(f'The maximum number of user-article interactions by any user is [{max_views_by_user}]')

In [ ]:
# df.groupby('article_id').agg({'email': 'count'}).sort_values('email', ascending=False).reset_index().rename(columns={'email': 'user_count'})

`2.` Explore and remove duplicate articles from the **df_content** dataframe.  

In [ ]:
# find and explore duplicate articles
df_content.nunique()

In [ ]:
df_content[df_content['doc_body'].duplicated()]

In [ ]:
# let's drop all the nans and again see the duplicated rows
df_content[(df_content['doc_body'].duplicated()) & (df_content['doc_body'] != np.nan)].dropna(subset='doc_body')

In [ ]:
df_content[df_content['doc_description'].duplicated()]

In [ ]:
# let's drop all the nans and again see the duplicated rows
df_content[(df_content['doc_description'].duplicated()) & (df_content['doc_description'] != np.nan)].dropna(subset='doc_description')

It looks like most of these are recurring weekly articles that have same description but fresh content

In [ ]:
df_content[df_content['doc_full_name'].duplicated()]

In [ ]:
df_content[df_content['article_id'].duplicated()]

In [ ]:
df_content[df_content['doc_full_name'].duplicated()].index.tolist() == df_content[df_content['article_id'].duplicated()].index.tolist()

From the cells above, we can see that there are in total 5 duplicates that can be found either on article_id (which is our primary key) or on doc_full_name.

In [ ]:
# remove any rows that have the same article_id - only keep the first
df_content = df_content.drop_duplicates(subset='article_id', keep='first', ignore_index=True)
df_content

`3.` Use the cells below to find:

**a.** The number of unique articles that have an interaction with a user.  
**b.** The number of unique articles in the dataset (whether they have any interactions or not).<br>
**c.** The number of unique users in the dataset. (excluding null values) <br>
**d.** The number of user-article interactions in the dataset.

In [ ]:
# the number of unique articles that have at least one interaction
unique_articles = df['article_id'].nunique()

# the number of unique articles on the IBM platform
total_articles = df_content['article_id'].nunique()

# the number of unique users
unique_users = df['email'].nunique()

# the number of user-article interactions
user_article_interactions = df.shape[0]

`4.` Use the cells below to find the most viewed **article_id**, as well as how often it was viewed.  After talking to the company leaders, the `email_mapper` function was deemed a reasonable way to map users to ids.  There were a small number of null values, and it was found that all of these null values likely belonged to a single user (which is how they are stored using the function below).

In [ ]:
# the most viewed article in the dataset as a string with one value following the decimal
most_viewed_article_id = str(df['article_id'].value_counts().index[np.where(df['article_id'].value_counts() == df['article_id'].value_counts().max())[0][0]])
# the most viewed article in the dataset was viewed how many times?
max_views = df['article_id'].value_counts()[df['article_id'].value_counts().index == df['article_id'].value_counts().index[np.where(df['article_id'].value_counts() == df['article_id'].value_counts().max())[0][0]]].values[0]

In [ ]:
# no need to change the code here - this will be helpful for later parts of the notebook. run this cell to map the user email to a user_id column and remove the email column

def email_mapper(email_series):
    coded_dict, cter, email_encoded = dict(), 1, []
    
    for val in email_series:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        email_encoded.append(coded_dict[val])
    return email_encoded

df['user_id'] = email_mapper(df['email'])
df = df.drop(columns='email')
df.head()

In [ ]:
sol_1_dict = {
    '`50% of individuals have _____ or fewer interactions.`': median_val,
    '`The total number of user-article interactions in the dataset is ______.`': user_article_interactions,
    '`The maximum number of user-article interactions by any 1 user is ______.`': max_views_by_user,
    '`The most viewed article in the dataset was viewed _____ times.`': max_views,
    '`The article_id of the most viewed article is ______.`': most_viewed_article_id,
    '`The number of unique articles that have at least 1 rating ______.`': unique_articles,
    '`The number of unique users in the dataset is ______`': unique_users,
    '`The number of unique articles on the IBM platform`': total_articles
}

### <a class="anchor" id="Rank">Part II: Rank-Based Recommendations</a>

Unlike in the earlier lessons, we don't actually have ratings for whether a user liked an article or not.  We only know that a user has interacted with an article.  In these cases, the popularity of an article can really only be based on how often an article was interacted with.

`1.` Fill in the function below to return the **n** top articles ordered with most interactions as the top. Test your function using the tests below.

In [ ]:
def get_top_articles(n, df):
    '''
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article titles
    '''

    return df.groupby(by=['article_id', 'title'], as_index=False).agg({'user_id':'count'}).nlargest(n, 'user_id')['title'].tolist()

def get_top_article_ids(n, df):
    '''
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article titles
    '''
 
    return df.groupby(by=['article_id', 'title'], as_index=False).agg({'user_id':'count'}).nlargest(n, 'user_id')['article_id'].tolist()

In [ ]:
print(get_top_articles(10, df))
print(get_top_article_ids(10, df))

In [ ]:
# Test your function by returning the top 5, 10, and 20 articles
top_5 = get_top_articles(5, df)
top_10 = get_top_articles(10, df)
top_20 = get_top_articles(20, df)

### <a class="anchor" id="User-User">Part III: User-User Based Collaborative Filtering</a>


`1.` Use the function below to reformat the **df** dataframe to be shaped with users as the rows and articles as the columns.  

* Each **user** should only appear in each **row** once.


* Each **article** should only show up in one **column**.  


* **If a user has interacted with an article, then place a 1 where the user-row meets for that article-column**.  It does not matter how many times a user has interacted with the article, all entries where a user has interacted with an article should be a 1.  


* **If a user has not interacted with an item, then place a zero where the user-row meets for that article-column**. 

Use the tests to make sure the basic structure of your matrix matches what is expected by the solution.

In [ ]:
# create the user-article matrix with 1's and 0's

def create_user_item_matrix(df):
    '''
    INPUT:
    df - pandas dataframe with article_id, title, user_id columns
    
    OUTPUT:
    user_item - user item matrix 
    
    Description:
    Return a matrix with user ids as rows and article ids on the columns with 1 values where a user interacted with 
    an article and a 0 otherwise
    '''
    temp_df = df.groupby(by=['user_id', 'article_id'], as_index=False).agg({'title':'count'})
    temp_df['title'] = 1
    return pd.pivot_table(temp_df, columns='article_id', index='user_id', values='title').fillna(0).astype('int')

user_item = create_user_item_matrix(df)

In [ ]:
## Tests: You should just need to run this cell.  Don't change the code.
assert user_item.shape[0] == 5149, "Oops!  The number of users in the user-article matrix doesn't look right."
assert user_item.shape[1] == 714, "Oops!  The number of articles in the user-article matrix doesn't look right."
assert user_item.sum(axis=1)[1] == 36, "Oops!  The number of articles seen by user 1 doesn't look right."
print("You have passed our quick tests!  Please proceed!")

`2.` Complete the function below which should take a user_id and provide an ordered list of the most similar users to that user (from most similar to least similar).  The returned result should not contain the provided user_id, as we know that each user is similar to him/herself. Because the results for each user here are binary, it (perhaps) makes sense to compute similarity as the dot product of two users. 

Use the tests to test your function.

In [ ]:
similarity_matrix = user_item.dot(user_item.T).reset_index()

In [ ]:
def find_similar_users(user_id, similarity_matrix):
    '''
    INPUT:
    user_id - (int) a user_id
    similarity matrix - (pandas dataframe) dot product reflecting similarity of all user w.r.t. other users in the dataset
    OUTPUT:
    similar_users - (list) an ordered list where the closest users (largest dot product users) are listed first    
    '''
    # sort by similarity AND remove the own user's id AND create list of just the ids AND return a list of the users in order from most to least similar
    return similarity_matrix[similarity_matrix['user_id'] == user_id].drop(columns=['user_id', user_id]).T.sort_values(by=user_id - 1, ascending=False).index.tolist()

In [ ]:
# Do a spot check of your function
print("The 10 most similar users to user 1 are: {}".format(find_similar_users(1, similarity_matrix)[:10]))
print("The 5 most similar users to user 3933 are: {}".format(find_similar_users(3933, similarity_matrix)[:5]))
print("The 3 most similar users to user 46 are: {}".format(find_similar_users(46, similarity_matrix)[:3]))

`3.` Now that you have a function that provides the most similar users to each user, you will want to use these users to find articles you can recommend.  Complete the functions below to return the articles you would recommend to each user. 

In [ ]:
def get_article_names(article_ids, df=df):
    '''
    INPUT:
    article_ids - (list) a list of article ids
    df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
    article_names - (list) a list of article names associated with the list of article ids (this is identified by the title column)
    '''
    article_ids = np.array(article_ids).astype(float).tolist()
    temp_df = df[['article_id', 'title']].drop_duplicates()
    return temp_df[temp_df['article_id'].isin(article_ids)]['title'].tolist()

def get_user_articles(user_id, user_item=user_item):
    '''
    INPUT:
    user_id - (int) a user id
    user_item - (pandas dataframe) matrix of users by articles: 1's when a user has interacted with an article, 0 otherwise
    df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
    article_ids - (list) a list of the article ids seen by the user
    article_names - (list) a list of article names associated with the list of article ids (this is identified by the doc_full_name column in df_content)
    
    Description:
    Provides a list of the article_ids and article titles that have been seen by a user
    '''
    article_ids= user_item.iloc[user_id - 1][user_item.iloc[user_id -1 ] == 1].index.tolist()
    article_names = get_article_names(article_ids) 
    return np.array(article_ids).astype(str).tolist(), article_names


def user_user_recs(user_id, user_item=user_item, similarity_matrix=similarity_matrix, m=10):
    '''
    INPUT:
    user_item - (pandas dataframe) matrix of users by articles: 1's when a user has interacted with an article, 0 otherwise
    similarity matrix - (pandas dataframe) dot product reflecting similarity of all user w.r.t. other users in the dataset
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user
    
    Description:
    Loops through the users based on closeness to the input user_id For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    Users who are the same closeness are chosen arbitrarily as the 'next' user
    
    For the user where the number of recommended articles starts below m and ends exceeding m, the last items are chosen arbitrarily
    '''
    articles_read_by_user_id, _ = get_user_articles(user_id, user_item)
    similar_users = find_similar_users(user_id, similarity_matrix)
    recs = []

    for user in similar_users:
        articles_read_by_user, _ = get_user_articles(user, user_item)
        articles_read_by_user_filtered = [art_id for art_id in articles_read_by_user if art_id not in articles_read_by_user_id]
        recs.extend(articles_read_by_user_filtered)
        if len(recs) >= m:
            recs = recs[:m]
            break

    return np.array(recs).astype(str).tolist()

In [ ]:
# Check Results
get_article_names(user_user_recs(1))

In [ ]:
# Test your functions here - No need to change this code - just run this cell
assert set(get_article_names(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_article_names(['1320.0', '232.0', '844.0'])) == set(['housing (2015): united states demographic measures','self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_user_articles(20)[0]) == set(['1320.0', '232.0', '844.0'])
assert set(get_user_articles(20)[1]) == set(['housing (2015): united states demographic measures', 'self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook'])
assert set(get_user_articles(2)[0]) == set(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])
assert set(get_user_articles(2)[1]) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis'])
print("If this is all you see, you passed all of our tests!  Nice job!")

`4.` Now we are going to improve the consistency of the **user_user_recs** function from above.  

* Instead of arbitrarily choosing when we obtain users who are all the same closeness to a given user - choose the users that have the most total article interactions before choosing those with fewer article interactions.


* Instead of arbitrarily choosing articles from the user where the number of recommended articles starts below m and ends exceeding m, choose articles with the articles with the most total interactions before choosing those with fewer total interactions. This ranking should be  what would be obtained from the **top_articles** function you wrote earlier.

In [ ]:
def get_top_sorted_users(user_id, df=df, similarity_matrix=similarity_matrix):
    '''
    INPUT:
    user_id - (int)
    df - (pandas dataframe) df as defined at the top of the notebook 
    user_item - (pandas dataframe) matrix of users by articles: 1's when a user has interacted with an article, 0 otherwise

    OUTPUT:
    neighbors_df - (pandas dataframe) a dataframe with: 
        neighbor_id - is a neighbor user_id
        similarity - measure of the similarity of each user to the provided user_id
        num_interactions - the number of articles viewed by the user
                    
    Other Details - sort the neighbors_df by the similarity and then by number of interactions where highest of each is higher in the dataframe
    '''
    temp_df = df.groupby('user_id', as_index=False).agg({'article_id': 'count'})
    temp_df = temp_df[temp_df['user_id'] != user_id].reset_index(drop=True).rename(columns={'article_id': 'num_interactions'})

    neighbors_df = similarity_matrix[similarity_matrix['user_id'] == user_id].drop(columns=['user_id', user_id]).T.reset_index().rename(columns={'user_id': 'neighbor_id', user_id - 1: 'similarity'}).sort_values(by='similarity', ascending=False).reset_index(drop=True)

    neighbors_df = pd.merge(left=neighbors_df, right=temp_df, left_on='neighbor_id', right_on='user_id', how='inner')
    neighbors_df = neighbors_df.drop(columns='user_id').sort_values(by=['similarity', 'num_interactions'], ascending=False)
    
    return neighbors_df


def user_user_recs_part2(user_id, m=10):
    '''
    INPUT:
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user by article id
    rec_names - (list) a list of recommendations for the user by article title
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    * Choose the users that have the most total article interactions before choosing those with fewer article interactions.

    * Choose articles with the articles with the most total interactions before choosing those with fewer total interactions. 
   '''
    recs = []
    neighbors_df = get_top_sorted_users(user_id)
    for user in neighbors_df['neighbor_id']:
        article_ids, _ = get_user_articles(user)
        article_ids = df.astype(str)[df['article_id'].astype(str).isin(article_ids)].groupby('article_id', as_index=False).agg({'user_id': 'count'}).sort_values(by='user_id', ascending=False).reset_index(drop=True)['article_id'].tolist()
        if len(recs) < m:
            recs.extend(article_ids)
            recs = list(set(recs))
        else:
            recs = recs[:10]
            break

    rec_names = get_article_names(recs)
    
    
    return recs, rec_names

In [ ]:
# Quick spot check - don't change this code - just use it to test your functions
rec_ids, rec_names = user_user_recs_part2(20, 10)
print("The top 10 recommendations for user 20 are the following article ids:")
print(rec_ids)
print()
print("The top 10 recommendations for user 20 are the following article names:")
print(rec_names)

`5.` Use your functions from above to correctly fill in the solutions to the dictionary below.  Then test your dictionary against the solution.  Provide the code you need to answer each following the comments below.

In [ ]:
### Tests with a dictionary of results

user1_most_sim = get_top_sorted_users(1)['neighbor_id'].values[0]
user131_10th_sim = get_top_sorted_users(131)['neighbor_id'].values[9]

sol_5_dict = {
    'The user that is most similar to user 1.': user1_most_sim, 
    'The user that is the 10th most similar to user 131': user131_10th_sim,
}

`6.` If we were given a new user, which of the above functions would you be able to use to make recommendations?  Explain.  Can you think of a better way we might make recommendations?  Use the cell below to explain a better method for new users.

**Answer**

We can only use `get_top_articles` and `get_top_article_ids` functions to make recommendation to the new user as there is no history of articles read by this new user and this history is the core necessity of Colaborative-Filteraing (what we built till now).

Another way could be provide a random set of articles to user and ask which articles of all those provided would he/she like to read and can repeat this process until we have a certain (let's say 10) number of articles selected by the user.

`7.` Using your existing functions, provide the top 10 recommended articles you would provide for the a new user below.  You can test your function against our thoughts to make sure we are all on the same page with how we might make a recommendation.

In [ ]:
new_user = '0.0'

# What would your recommendations be for this new user '0.0'?  As a new user, they have no observed articles.
# Provide a list of the top 10 article ids you would give to 
new_user_recs = get_top_article_ids(10, df)
new_user_recs = np.array(new_user_recs).astype(str)

In [ ]:
assert set(new_user_recs) == set(['1314.0','1429.0','1293.0','1427.0','1162.0','1364.0','1304.0','1170.0','1431.0','1330.0']), "Oops!  It makes sense that in this case we would want to recommend the most popular articles, because we don't know anything about these users."

print("That's right!  Nice job!")

### <a class="anchor" id="Content-Recs">Part IV: Content Based Recommendations (EXTRA - NOT REQUIRED)</a>

Another method we might use to make recommendations is to perform a ranking of the highest ranked articles associated with some term.  You might consider content to be the **doc_body**, **doc_description**, or **doc_full_name**.  There isn't one way to create a content based recommendation, especially considering that each of these columns hold content related information.  

`1.` Use the function body below to create a content based recommender.  Since there isn't one right answer for this recommendation tactic, no test functions are provided.  Feel free to change the function inputs if you decide you want to try a method that requires more input values.  The input values are currently set with one idea in mind that you may use to make content based recommendations.  One additional idea is that you might want to choose the most popular recommendations that meet your 'content criteria', but again, there is a lot of flexibility in how you might make these recommendations.

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

In [ ]:
def make_content_recs():
    '''
    INPUT:
    
    OUTPUT:
    
    '''

`2.` Now that you have put together your content-based recommendation system, use the cell below to write a summary explaining how your content based recommender works.  Do you see any possible improvements that could be made to your function?  Is there anything novel about your content based recommender?

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

**Write an explanation of your content based recommendation system here.**

`3.` Use your content-recommendation system to make recommendations for the below scenarios based on the comments.  Again no tests are provided here, because there isn't one right answer that could be used to find these content based recommendations.

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

In [ ]:
# make recommendations for a brand new user


# make a recommendations for a user who only has interacted with article id '1427.0'



### <a class="anchor" id="Matrix-Fact">Part V: Matrix Factorization</a>

In this part of the notebook, you will build use matrix factorization to make article recommendations to the users on the IBM Watson Studio platform.

`1.` You should have already created a **user_item** matrix above in **question 1** of **Part III** above.  This first question here will just require that you run the cells to get things set up for the rest of **Part V** of the notebook. 

In [ ]:
# Load the matrix here
user_item_matrix = pd.read_pickle('data/user_item_matrix.p')
# quick look at the matrix
user_item_matrix.head()

`2.` In this situation, you can use Singular Value Decomposition from [numpy](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html) on the user-item matrix.  Use the cell to perform SVD, and explain why this is different than in the lesson.

In [ ]:
# Perform SVD on the User-Item Matrix Here
# use the built in to get the three matrices
u, s, vt = np.linalg.svd(user_item_matrix)

**Answer**
In the lesson we experienced a situation where we had `NULL` values in our dataset and hence had to use `FunkSVD` to perform Matrix Factorization. But contrary to the situation in the lesson, here we do not have any null vales in our dataset and hence we are able to perform `SVD`.

`3.` Now for the tricky part, how do we choose the number of latent features to use?  Running the below cell, you can see that as the number of latent features increases, we obtain a lower error rate on making predictions for the 1 and 0 values in the user-item matrix.  Run the cell below to get an idea of how the accuracy improves as we increase the number of latent features.

In [ ]:
num_latent_feats, sum_errs = np.arange(10,700+10,20), []

for k in num_latent_feats:
    # restructure with k latent features
    s_new, u_new, vt_new = np.diag(s[:k]), u[:, :k], vt[:k, :]
    
    # take dot product
    user_item_est = np.around(np.dot(np.dot(u_new, s_new), vt_new))
    
    # compute error for each prediction to actual value
    diffs = np.subtract(user_item_matrix, user_item_est)
    
    # total errors and keep track of them
    err = np.sum(np.sum(np.abs(diffs)))
    sum_errs.append(err)

accuracy = 1 - np.array(sum_errs)/df.shape[0]
fig = px.line(x=num_latent_feats, y=accuracy, width=1300)
fig.update_layout(
    xaxis={'visible': True, 'showticklabels': True},
    font_family='Input Mono',
    font_color='black',
    title_x=0.50,
    title_y=0.95,
    title_text='Train Set Accuracy vs. Number of Latent Features',
    xaxis_title='Number of Latent Features',
    yaxis_title='Train Set Accuracy',
    plot_bgcolor='rgba(255,255,255,255)',
    paper_bgcolor='rgba(255,255,255,255)')
fig.show()

`4.` From the above, we can't really be sure how many features to use, because simply having a better way to predict the 1's and 0's of the matrix doesn't exactly give us an indication of if we are able to make good recommendations.  Instead, we might split our dataset into a training and test set of data, as shown in the cell below.  

Use the code from question 3 to understand the impact on accuracy of the training and test sets of data with different numbers of latent features. Using the split below: 

* How many users can we make predictions for in the test set?  
* How many users are we not able to make predictions for because of the cold start problem?
* How many articles can we make predictions for in the test set?  
* How many articles are we not able to make predictions for because of the cold start problem?

In [ ]:
df_train = df.head(40000)
df_test = df.tail(5993)

def create_test_and_train_user_item(df_train, df_test):
    '''
    INPUT:
    df_train - training dataframe
    df_test - test dataframe
    
    OUTPUT:
    user_item_train - a user-item matrix of the training dataframe 
                      (unique users for each row and unique articles for each column)
    user_item_test - a user-item matrix of the testing dataframe 
                    (unique users for each row and unique articles for each column)
    test_idx - all of the test user ids
    test_arts - all of the test article ids
    
    '''
    user_item_train = create_user_item_matrix(df_train)
    user_item_test = create_user_item_matrix(df_test)

    return user_item_train, user_item_test, user_item_test.index, user_item_test.columns

user_item_train, user_item_test, test_idx, test_arts = create_test_and_train_user_item(df_train, df_test)

In [ ]:
sol_4_dict = {
    'How many users can we make predictions for in the test set?': len(set(user_item_test.index).intersection(set(user_item_train.index))), 
    'How many users in the test set are we not able to make predictions for because of the cold start problem?': len(set(user_item_test.index)) - len(set(user_item_test.index).intersection(set(user_item_train.index))), 
    'How many articles can we make predictions for in the test set?': len(set(user_item_test.columns).intersection(user_item_train.columns)),
    'How many articles in the test set are we not able to make predictions for because of the cold start problem?': len(set(user_item_test.columns)) - len(set(user_item_test.columns).intersection(user_item_train.columns))
}

`5.` Now use the **user_item_train** dataset from above to find U, S, and V transpose using SVD. Then find the subset of rows in the **user_item_test** dataset that you can predict using this matrix decomposition with different numbers of latent features to see how many features makes sense to keep based on the accuracy on the test data. This will require combining what was done in questions `2` - `4`.

Use the cells below to explore how well SVD works towards making predictions for recommendations on the test data.  

In [ ]:
# fit SVD on the user_item_train matrix
# fit svd similar to above then use the cells below
u_train, s_train, vt_train = np.linalg.svd(user_item_train)

In [ ]:
u_train.shape, s_train.shape, vt_train.shape

In [ ]:
# Find the subset in the test data for which we can make predictions
u_test = u_train[user_item_train.index.isin(user_item_test.index), :]
vt_test = vt_train[:, user_item_train.columns.isin(user_item_test.columns)]

u_test.shape, vt_test.shape

In [ ]:
user_item_test_subset = user_item_test.reset_index()
user_item_test_subset = user_item_test_subset[user_item_test_subset['user_id'].isin(set(user_item_test.index).intersection(set(user_item_train.index)))]
user_item_test_subset = user_item_test_subset[[col for col in user_item_test_subset if col in set(user_item_test.columns).intersection(user_item_train.columns)]]

In [ ]:
num_latent_feats, sum_errs = np.arange(10,700+10,20), []

for k in num_latent_feats:
    # restructure with k latent features
    s_test_new, u_test_new, vt_test_new = np.diag(s_train[:k]), u_test[:, :k], vt_test[:k, :]
    
    # take dot product
    user_item_test_est = np.around(np.dot(np.dot(u_test_new, s_test_new), vt_test_new))
    
    # compute error for each prediction to actual value
    diffs = np.subtract(user_item_test_subset, user_item_test_est)
    
    # total errors and keep track of them
    err = np.sum(np.sum(np.abs(diffs)))
    sum_errs.append(err)
    
accuracy = 1 - np.array(sum_errs)/df.shape[0]
fig = px.line(x=num_latent_feats, y=accuracy, width=1300)
fig.update_layout(
    xaxis={'visible': True, 'showticklabels': True},
    font_family='Input Mono',
    font_color='black',
    title_x=0.50,
    title_y=0.95,
    title_text='Test Set Accuracy vs. Number of Latent Features',
    xaxis_title='Number of Latent Features',
    yaxis_title='Test Set Accuracy',
    plot_bgcolor='rgba(255,255,255,255)',
    paper_bgcolor='rgba(255,255,255,255)')
fig.show()

`6.` Use the cell below to comment on the results you found in the previous question. Given the circumstances of your results, discuss what you might do to determine if the recommendations you make with any of the above recommendation systems are an improvement to how users currently find articles? 

**Answer**

Okay, so the plot we generated above using the test data is contrary to what we saw in the plot that was generated using the train data as accuracy decreases with increase in the number of latent features for test data. The plausible reason for this decrease is such a small size of our test set (20 samples) which causes the trained model to NOT generalize well. We should keep less number of latent features in our train set to prevent overfitting of the training set and as a result a more generalized results on test set.

Due to this small number of observations in out test set, it does not make any sense for us to test the accuracy of our model in an offline mode and instead we should perform online testing to determine how well our predictions are.

<a id='conclusions'></a>
### Extras
Using your workbook, you could now save your recommendations for each user, develop a class to make new predictions and update your results, and make a flask app to deploy your results.  These tasks are beyond what is required for this project.  However, from what you learned in the lessons, you certainly capable of taking these tasks on to improve upon your work here!